## Cluster Analysis with Multinomial EM (Text Clustering)

#### In the K-means clustering algorithm we forced documents to be assigned to one cluster. This is known as hard clustering. This approach in not really accurate when working with language.  When someone writes about a topic they use a certain terminology to describe that topic. We assume that when someone else writes about the same topic they are likely to use the same terminology or words. But words may also be used across multiple topics and a clinical note may describe multiple topics. It may be more appropriate to assign words and documents to multiple topics (or clusters) with a certain probability based on their use.

#### This example illustrates how to group a set of different clinical notes based on their topics and the terminology used to describe the topics. We will model our clinical notes as a collection of unigram and bigram words. Specifically, we will represent each clinical note as a feature set of unigram and bigram frequencies found in the clinical note. We will use a matrix where each row will represent a clinical note and each column a feature (i.e. distinct unigram or bigram).

### Mixture of Multinomial Distributions

#### Text is best represented as a mixture of multinomial distributions where each topic has a particular multinomial distribution associated with it and each document in a mixture of topics. 

#### Formally, let $p(c)\space=\space\pi_c$ be the prior probability of a document containing topic c, and each topic c is represented as a multinomial distribution $p(D_i|c)$ with parameters $\mu_{jc}$, then each document becomes a mixture over topics as

#### $p(D_i)\space=\space \displaystyle\sum_{c=1}^{n_c} p(D_i|c)p(c) = \displaystyle\sum_{c=1}^{n_c} \pi_c \prod_{j=1}^{n_w}\mu_{jc}^{T_{ij}}$

### Expectation Maximization for Mixtures of Multinomials

#### The expectation maximization algorithm will allow us to fit a multinomial mixture model to our data. Our goal is to identify which documents belong to which topics and what words (unigrams and bigrams) are used to describe the topic. 

#### 1. E-Step. Compute the expectation that document $D_i$ belongs to topic (cluster) $c$

####         $\gamma_{ic} \propto \pi_c \displaystyle\prod_{j = 1}^{n_w} \mu_{jc}^{T_{ij}}$

#### $Note: \space normalize \space expectations \space over \space c$

#### where,
#### $\pi_c \equiv prior{\space}probability{\space}of{\space}document{\space}containing{\space}topic{\space}c$
#### $\mu_{jc} \equiv probability{\space}of{\space}w_j{\space}in{\space}topic{\space}c$
#### $T_{ij} \equiv count \space of \space w_j \space in \space topic \space c$


#### 2. M-Step. Update the mixture parameters. 

#### $\mu_{jc} = \frac{\sum_{i = 1}^{n_d} \gamma_{ic}T_{ij}}{\sum_{i = 1}^{n_d}\sum_{l = 1}^{n_w} \gamma_{ic}T_{il}} \equiv probability \space of \space a \space word \space being \space w_j \space in \space topic \space (cluster) \space c$ 

#### $\pi_c = \frac{1}{n_d} \displaystyle \sum_{i = 1}^{n_d} \gamma_{ic} \equiv prior \space probability \space of \space each \space cluster$

#### $Note: \space normalize \space priors \space uniformly,\space initialize \space \mu's \space to \space multinomial \space generated \space from \space uniform \space dirichlet \space distribution \space such \space that \sum_{j = 1}^{n_w} \mu_{jc} = 1$

### Implementation

#### Our implementation will consist of miipulating 4 matricies as described below.

![title](./images/MultinomialEMClustering.png)


### Environment Setup

#### First we will import some Python packages that we will use.

In [1]:
import nltk
import re
import pandas as pd
import numpy as np 
import numpy.matlib as ml
import pickle as pkl
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Syntactic NLP Processing

#### We need a function to tokenize our text and remove noise like dates, ages, etc.

In [2]:
def tokenize(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = [ token for token in tokens if re.search('(^[a-zA-Z]+$)', token) ]
    return filtered_tokens

### Retrieving our Corpus

#### Let's pull in our corpus that we had serialized out to disk.  

In [3]:
file = open('differential-corpus.pkl','rb')
corpus = pkl.load(file)
file.close()

### More Syntactic Processing

#### We will want to get ride of stop words that are essentially noise. 

In [4]:
cachedStopWords = stopwords.words("english") + ['year', 'old', 'man', 'woman', 'ap', 'am', 'pm', 'portable', 'pa', 'lat', 'admitting', 'diagnosis', 'lateral']
print(cachedStopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

### Generate Document-Term Frequency Counts

#### In this step we tokenize our text and remove stop words in addition to generating our frequency counts.

#### 1) how many documents are we working with and how many features (unigrams & bigrams)?

#### 2) Can you figure out what max_df and min_df is doing to our feature count?

In [5]:
corpusList = list(corpus.values())
labels = list(corpus.keys())
rank = list(range(len(labels)))

cv = CountVectorizer(lowercase=True, max_df=0.80, max_features=None, min_df=0.033,
                     ngram_range=(2, 2), preprocessor=None, stop_words=cachedStopWords,
                     strip_accents=None, tokenizer=tokenize, vocabulary=None)
SparseT = cv.fit_transform(corpusList)
print("The dimensions of our document-term matrix")
print(SparseT.shape)
print()

The dimensions of our document-term matrix
(1500, 291)



### Feature Set

#### Let's take a look at our feature set.

#### 1) Do we have a lot of noise in our features?

In [6]:
lexicon = cv.get_feature_names()
print (lexicon)
print()

['abdomen contrast', 'abdomen pelvis', 'acute sob', 'adrenal glands', 'air space', 'also noted', 'amt final', 'amt underlying', 'aortic stenosis', 'approximately cm', 'atelectasis left', 'axial images', 'bilateral pleural', 'bilateral pulmonary', 'bone windows', 'cad chf', 'cardiac mediastinal', 'cardiac silhouette', 'catheter terminates', 'catheter tip', 'cc optiray', 'central line', 'central venous', 'change final', 'chest clip', 'chest compared', 'chest comparison', 'chest contrast', 'chest ct', 'chest different', 'chest history', 'chest indication', 'chest obtained', 'chest pain', 'chest physician', 'chest port', 'chest radiograph', 'chest single', 'chest w', 'chf final', 'chf reason', 'chf underlying', 'clinical indication', 'cm carina', 'compared previous', 'compared prior', 'comparison chest', 'comparison findings', 'comparison made', 'condition acute', 'condition cad', 'condition chf', 'condition copd', 'condition critical', 'condition reason', 'condition sob', 'condition yo', 

## Define EM Algorithm (E-Step & M-Step)

#### Now let's define our EM Algorithm.

#### 1) In the ExpD function why are we multiplying by $1x10^2 \space ?$  

In [7]:
  
def ExpD(T, Mu, Pi):    
    C_n = Pi.shape[1]
    D_n = T.shape[0]
    Gamma = ml.zeros((D_n, C_n))
    Gamma.astype('float64')
    for c in range(0, C_n):
        Gamma[:, c] = Pi[0][c] * ((Mu[:,c].A[:,0]*1e2)**T).prod(1)
    Gamma = Gamma / Gamma.sum(axis=1)
    return Gamma
    
def updateMu(T, Gamma):    
    C_n = Gamma.shape[1]
    W_n = T.shape[1]
    Mu = ml.zeros((W_n, C_n))
    for c in range(0, C_n):
        numerator = sum(np.multiply(Gamma[:,c],T)).T
        demoninator = sum(np.multiply(Gamma[:,c],T).sum(1))
        Mu[:,c] = numerator / demoninator
    return Mu
    
def updatePi(Gamma):    
    D_n = Gamma.shape[0]
    Pi = sum(Gamma) / D_n
    return Pi.A
    

## Let's Run it !

#### Let's start with 10 topics (clusters) and we will interate 100 times. EM converges quickly.

#### 1) Can you determine at what iteration we are starting to reach convergence?

In [8]:
T = SparseT.todense()
D_n, W_n = T.shape
C_n = 10
Pi = ml.repmat(1/C_n, 1, C_n)
Mu = ml.mat(np.random.dirichlet(np.ones(W_n), C_n).T)
for i in range(1,101):
    print('Iteration: ' + str(i)) 
    Gamma = ExpD(T, Mu, Pi)
    print(Gamma.sum(0))
    Mu = updateMu(T, Gamma)
    Pi = updatePi(Gamma)


Iteration: 1
[[ 131.87539751   59.87130159  150.12928566   88.94851851  123.10390427
    37.38759415  386.72844768  171.21782129  108.25826464  242.47946471]]
Iteration: 2
[[  90.20138752  136.00621825  129.98623656  137.63747296  164.20144995
    45.09499528  333.80117986   71.9563737    43.44636717  347.66831876]]
Iteration: 3
[[  83.45265689  163.21417411  123.23276978  149.80156645  183.11456476
    48.10432957  294.98946615   52.82149208   28.53091354  372.73806666]]
Iteration: 4
[[  79.17203752  170.78629359  119.80822283  175.40877977  197.85144575
    49.46404865  266.0839504    51.23731995   27.29439059  362.89351095]]
Iteration: 5
[[  78.10624218  182.0361448   119.00698618  198.46184124  208.73778141
    48.23591676  241.53839866   51.251877     26.47562518  346.14918659]]
Iteration: 6
[[  77.66991382  191.24899471  118.67026169  211.92959435  217.24041169
    47.33867771  229.82324164   51.26312086   26.2016142   328.61416933]]
Iteration: 7
[[  77.51562939  200.13472144  11

Iteration: 53
[[  77.24134453  248.55467544  118.01141909  234.18031536  266.68225114
    43.70237372  172.00313444   51.28331421   26.03455799  262.3066141 ]]
Iteration: 54
[[  77.24401456  247.72340863  118.01238552  234.78839995  266.66044626
    43.68612281  171.99491572   51.28331937   26.0380932   262.56889399]]
Iteration: 55
[[  77.2472433   246.6401407   118.01556658  235.49525066  266.65099795
    43.66939624  171.98700893   51.28331599   26.04431752  262.96676213]]
Iteration: 56
[[  77.25321751  245.85039434  118.02004931  235.94744187  266.65629278
    43.65053892  171.97918278   51.28330662   26.05217342  263.30740245]]
Iteration: 57
[[  77.2623255   245.42553261  118.02205245  236.26938916  266.65487497
    43.62750964  171.96605079   51.28329119   26.05654849  263.4324252 ]]
Iteration: 58
[[  77.26885516  245.17714488  118.02127614  236.55887318  266.6111503
    43.59708867  171.77020626   51.28327267   26.05897974  263.65315301]]
Iteration: 59
[[  77.27230204  245.019989

## Examination of Clusters and Terminology

#### Let's take a look at the top cluster for each clinical note and the top 20 words to distinguish this topic.

#### 1) Is there noise in the terminology? If there is how can we get ride of it?


In [9]:
clusters = Gamma.argmax(1).A
clusters = [i[0] for i in clusters]
clinicalDocuments = { 'labels': labels, 'rank': rank, 'corpus': corpusList, 'cluster': clusters }
frame = pd.DataFrame(clinicalDocuments, index = clusters , columns = ['rank', 'labels', 'corpus', 'cluster'])
grouped = frame['rank'].groupby(frame['cluster'])
topWords = Mu.T.argsort()[:, ::-1]
for i in range(C_n):
    n = len(frame.ix[i]['labels'].values.tolist())
    print('Cluster %d (%d):,' % (i+1, n), end='')
    for label in frame.ix[i]['labels'].values.tolist():
        print(' %s,' % label, end='')
    print()
    print()
    print('           Words:', end='')
    for indice in list(topWords[i, :20].A[0]):
        print(' %s (%.5f)' % (lexicon[indice], Mu[indice,i]), end=',')
    print()
    print()
    print()

Cluster 1 (78):, PNA3, PNA4, PNA5, PNA6, PNA13, PNA14, PNA15, PNA49, PNA58, PNA59, PNA60, PNA74, PNA75, PNA76, PNA126, PNA129, PNA130, PNA131, PNA145, PNA146, PNA291, PNA293, PNA294, PNA315, PNA316, PNA317, PNA328, PNA329, PNA340, PNA348, PNA349, PNA350, PNA388, PNA389, PNA421, PNA422, PNA423, PNA439, PNA440, PNA441, CHF49, CHF50, CHF51, CHF124, CHF136, CHF137, CHF144, CHF150, CHF223, CHF224, CHF255, CHF256, CHF268, CHF279, CHF280, CHF301, CHF302, CHF303, CHF483, COPD49, COPD50, COPD51, COPD124, COPD136, COPD137, COPD144, COPD150, COPD223, COPD224, COPD255, COPD256, COPD268, COPD279, COPD280, COPD301, COPD302, COPD303, COPD483,

           Words: ct abdomen (0.04906), ct pelvis (0.04513), iv contrast (0.04490), date ct (0.03310), ct chest (0.02361), abdomen contrast (0.02337), field view (0.02245), pleural effusions (0.02187), contrast ct (0.02129), pelvis contrast (0.02106), contrast clip (0.02083), bilateral pleural (0.02025), radiology ct (0.01898), contrast reason (0.01666), report

Cluster 10 (264):, PNA66, PNA102, PNA120, PNA132, PNA134, PNA138, PNA142, PNA154, PNA157, PNA160, PNA174, PNA175, PNA179, PNA184, PNA192, PNA207, PNA209, PNA210, PNA211, PNA238, PNA242, PNA243, PNA260, PNA269, PNA273, PNA277, PNA282, PNA290, PNA296, PNA298, PNA305, PNA312, PNA322, PNA330, PNA331, PNA344, PNA353, PNA355, PNA377, PNA384, PNA385, PNA393, PNA394, PNA401, PNA402, PNA407, PNA411, PNA413, PNA415, PNA420, PNA429, PNA430, PNA432, PNA438, PNA452, PNA458, PNA459, PNA460, PNA469, PNA478, PNA486, PNA491, PNA493, PNA494, CHF4, CHF8, CHF14, CHF20, CHF21, CHF23, CHF31, CHF33, CHF34, CHF42, CHF62, CHF70, CHF77, CHF81, CHF86, CHF97, CHF100, CHF102, CHF103, CHF107, CHF109, CHF110, CHF111, CHF131, CHF138, CHF141, CHF142, CHF143, CHF148, CHF149, CHF155, CHF156, CHF162, CHF163, CHF164, CHF167, CHF171, CHF181, CHF216, CHF218, CHF221, CHF226, CHF228, CHF229, CHF230, CHF243, CHF244, CHF247, CHF251, CHF253, CHF254, CHF257, CHF258, CHF259, CHF273, CHF275, CHF295, CHF304, CHF314, CHF315, CHF316, 

### Soft Clustering Document Examination

#### 1) Can you find clinical notes that belong to more than 1 cluster ?


In [10]:
N, C = Gamma.shape
for i in range(N):
    print('%s: ' % labels[i], end='')
    for j in range(C):
        print('C%d (%.7f): ' % (j+1, Gamma[i, j]), end='')
    print()
    print()
print()

PNA1: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA2: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA3: C1 (1.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA4: C1 (1.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA5: C1 (1.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA6: C1 (1.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (

PNA69: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA70: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA71: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA72: C1 (0.0000000): C2 (0.9883307): C3 (0.0000000): C4 (0.0116693): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA73: C1 (0.0000000): C2 (0.9883307): C3 (0.0000000): C4 (0.0116693): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA74: C1 (1.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000):

PNA139: C1 (0.0000000): C2 (0.0000035): C3 (0.0000000): C4 (0.9982887): C5 (0.0017078): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA140: C1 (0.0000000): C2 (0.9915179): C3 (0.0000000): C4 (0.0000309): C5 (0.0000152): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0084360): 

PNA141: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA142: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0992946): C6 (0.0000000): C7 (0.0302815): C8 (0.0000000): C9 (0.0000000): C10 (0.8704239): 

PNA143: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (1.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA144: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.9868802): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.000

PNA210: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

PNA211: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

PNA212: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0094664): C9 (0.9905336): C10 (0.0000000): 

PNA213: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0094664): C9 (0.9905336): C10 (0.0000000): 

PNA214: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA215: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.000

PNA281: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA282: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

PNA283: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000836): C5 (0.9999164): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA284: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0344899): C5 (0.9062035): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0593066): 

PNA285: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (1.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA286: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (1.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.000

...

COPD170: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.9999583): C6 (0.0000000): C7 (0.0000049): C8 (0.0000000): C9 (0.0000000): C10 (0.0000369): 

COPD171: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000017): C5 (0.0270200): C6 (0.0000000): C7 (0.0022017): C8 (0.0000000): C9 (0.0000000): C10 (0.9707766): 

COPD172: C1 (0.0000000): C2 (0.0991899): C3 (0.0000000): C4 (0.9007964): C5 (0.0000137): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD173: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0001392): C5 (0.9998607): C6 (0.0000000): C7 (0.0000001): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD174: C1 (0.0000000): C2 (0.9999485): C3 (0.0000000): C4 (0.0000215): C5 (0.0000280): C6 (0.0000000): C7 (0.0000020): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD175: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 

COPD246: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD247: C1 (0.0000000): C2 (0.0001205): C3 (0.0000000): C4 (0.0000102): C5 (0.0000000): C6 (0.0000000): C7 (0.0000711): C8 (0.0000000): C9 (0.0000000): C10 (0.9997982): 

COPD248: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.9984134): C5 (0.0012680): C6 (0.0000000): C7 (0.0002560): C8 (0.0000000): C9 (0.0000000): C10 (0.0000626): 

COPD249: C1 (0.0000000): C2 (0.9735539): C3 (0.0000000): C4 (0.0262086): C5 (0.0000000): C6 (0.0000000): C7 (0.0002375): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD250: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (1.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD251: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000550): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 

COPD324: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD325: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD326: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0005278): C8 (0.0000000): C9 (0.0000000): C10 (0.9994722): 

COPD327: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0008071): C8 (0.0000000): C9 (0.0000000): C10 (0.9991929): 

COPD328: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000176): C8 (0.0000000): C9 (0.0000000): C10 (0.9999824): 

COPD329: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 

COPD400: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD401: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0001299): C8 (0.0000000): C9 (0.0000000): C10 (0.9998701): 

COPD402: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.9946740): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0053260): 

COPD403: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD404: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD405: C1 (0.0000000): C2 (1.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 

COPD478: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD479: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.1670706): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.8329294): 

COPD480: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0039881): C5 (0.9960119): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD481: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD482: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD483: C1 (0.7437627): C2 (0.2562373): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 

### Soft Clustering Term Examination

#### 1) Can you find terms that belong to more than 1 cluster ?

In [11]:
W, C = Mu.shape
for i in range(W):
    print('%s: ' % lexicon[i], end='')
    for j in range(C):
        print('C%d (%.5f): ' % (j+1, Mu[i, j]), end='')
    print()
    print()
print()

abdomen contrast: C1 (0.02337): C2 (0.00000): C3 (0.00000): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00393): C9 (0.00149): C10 (0.00000): 

abdomen pelvis: C1 (0.01065): C2 (0.00000): C3 (0.00000): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00786): C9 (0.01643): C10 (0.00000): 

acute sob: C1 (0.00000): C2 (0.00137): C3 (0.00156): C4 (0.00116): C5 (0.00229): C6 (0.00000): C7 (0.00045): C8 (0.00000): C9 (0.00000): C10 (0.00790): 

adrenal glands: C1 (0.01018): C2 (0.00000): C3 (0.00311): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00236): C9 (0.00972): C10 (0.00000): 

air space: C1 (0.00000): C2 (0.00102): C3 (0.00062): C4 (0.00182): C5 (0.00154): C6 (0.00426): C7 (0.00000): C8 (0.00275): C9 (0.00299): C10 (0.00709): 

also noted: C1 (0.00231): C2 (0.00244): C3 (0.00156): C4 (0.00212): C5 (0.00067): C6 (0.00000): C7 (0.00000): C8 (0.00510): C9 (0.00000): C10 (0.00024): 

amt final: C1 (0.00625): C2 (0.00000): C3 (0.00995): C4 (0

contrast ct: C1 (0.02129): C2 (0.00000): C3 (0.00389): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00707): C9 (0.00149): C10 (0.00000): 

contrast final: C1 (0.00648): C2 (0.00000): C3 (0.00731): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00589): C9 (0.01196): C10 (0.00000): 

contrast optiray: C1 (0.01319): C2 (0.00000): C3 (0.02209): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.02983): C9 (0.00598): C10 (0.00000): 

contrast reason: C1 (0.01666): C2 (0.00000): C3 (0.00000): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.01022): C9 (0.01940): C10 (0.00000): 

coronary artery: C1 (0.00301): C2 (0.00272): C3 (0.00560): C4 (0.00246): C5 (0.00438): C6 (0.00000): C7 (0.00214): C8 (0.00079): C9 (0.00299): C10 (0.00156): 

costophrenic angle: C1 (0.00000): C2 (0.00407): C3 (0.00000): C4 (0.00409): C5 (0.00397): C6 (0.00000): C7 (0.00180): C8 (0.00000): C9 (0.00000): C10 (0.00208): 

could represent: C1 (0.00000): C2 (0.0004

left base: C1 (0.00000): C2 (0.00298): C3 (0.00016): C4 (0.00169): C5 (0.00342): C6 (0.00426): C7 (0.00181): C8 (0.00000): C9 (0.00747): C10 (0.00197): 

left lower: C1 (0.00555): C2 (0.00279): C3 (0.01384): C4 (0.03136): C5 (0.01739): C6 (0.00426): C7 (0.01152): C8 (0.00628): C9 (0.00149): C10 (0.00851): 

left lung: C1 (0.00185): C2 (0.00241): C3 (0.00482): C4 (0.00883): C5 (0.00121): C6 (0.00000): C7 (0.00673): C8 (0.00000): C9 (0.00000): C10 (0.00243): 

left pleural: C1 (0.00046): C2 (0.00672): C3 (0.00638): C4 (0.01028): C5 (0.00740): C6 (0.02131): C7 (0.00882): C8 (0.00000): C9 (0.00299): C10 (0.00945): 

left retrocardiac: C1 (0.00000): C2 (0.00201): C3 (0.00000): C4 (0.00212): C5 (0.00538): C6 (0.00213): C7 (0.00068): C8 (0.00000): C9 (0.00000): C10 (0.00293): 

left side: C1 (0.00069): C2 (0.00136): C3 (0.00223): C4 (0.00053): C5 (0.00130): C6 (0.01065): C7 (0.00114): C8 (0.00079): C9 (0.00293): C10 (0.00000): 

left sided: C1 (0.00255): C2 (0.00174): C3 (0.00124): C4 (0.0035

pulmonary edema: C1 (0.00116): C2 (0.02330): C3 (0.00529): C4 (0.00786): C5 (0.01315): C6 (0.00000): C7 (0.02353): C8 (0.00079): C9 (0.01420): C10 (0.05409): 

pulmonary vascular: C1 (0.00000): C2 (0.00409): C3 (0.00000): C4 (0.00264): C5 (0.00304): C6 (0.01278): C7 (0.00406): C8 (0.00000): C9 (0.00000): C10 (0.00497): 

pulmonary vasculature: C1 (0.00000): C2 (0.00905): C3 (0.00047): C4 (0.00391): C5 (0.00285): C6 (0.00000): C7 (0.00310): C8 (0.00000): C9 (0.00000): C10 (0.00077): 

radiograph chest: C1 (0.00000): C2 (0.00292): C3 (0.00000): C4 (0.00135): C5 (0.00144): C6 (0.00000): C7 (0.00371): C8 (0.00000): C9 (0.00000): C10 (0.00347): 

radiology ct: C1 (0.01898): C2 (0.00000): C3 (0.00187): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.02357): C9 (0.01940): C10 (0.00000): 

reason assess: C1 (0.00000): C2 (0.00371): C3 (0.00156): C4 (0.00000): C5 (0.00211): C6 (0.00000): C7 (0.00113): C8 (0.00000): C9 (0.00000): C10 (0.00845): 

reason chf: C1 (0.00046): C2 (0.008

upper zone: C1 (0.00000): C2 (0.01020): C3 (0.00000): C4 (0.00221): C5 (0.00501): C6 (0.00426): C7 (0.00226): C8 (0.00000): C9 (0.00000): C10 (0.00083): 

upright chest: C1 (0.00000): C2 (0.00547): C3 (0.00031): C4 (0.00272): C5 (0.00000): C6 (0.00000): C7 (0.00045): C8 (0.00000): C9 (0.00000): C10 (0.00232): 

vascular congestion: C1 (0.00000): C2 (0.00249): C3 (0.00000): C4 (0.00126): C5 (0.00230): C6 (0.01065): C7 (0.00361): C8 (0.00000): C9 (0.00000): C10 (0.00452): 

vascular engorgement: C1 (0.00000): C2 (0.00205): C3 (0.00000): C4 (0.00169): C5 (0.00138): C6 (0.00000): C7 (0.00173): C8 (0.00000): C9 (0.00000): C10 (0.00369): 

venous catheter: C1 (0.00139): C2 (0.00000): C3 (0.00016): C4 (0.00231): C5 (0.00292): C6 (0.00000): C7 (0.00553): C8 (0.00196): C9 (0.00000): C10 (0.00591): 

view chest: C1 (0.00000): C2 (0.00334): C3 (0.00000): C4 (0.00490): C5 (0.00521): C6 (0.02344): C7 (0.00769): C8 (0.00000): C9 (0.00000): C10 (0.00117): 

view contrast: C1 (0.01180): C2 (0.00000): 